In [ ]:
%matplotlib inline
from pathlib import Path
path = Path('data')

In [ ]:
import pandas as pd


fname = path.joinpath('challenger_path.csv')
df = pd.read_csv(fname, names=['lon', 'lat'])
df.head()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap


def make_basemap(projection='robin', figsize=(9, 9), resolution='c'):
    fig, ax = plt.subplots(figsize=figsize)
    m = Basemap(projection=projection, resolution=resolution,
                lon_0=0, ax=ax)
    m.drawcoastlines()
    m.fillcontinents(color='0.85')
    return fig, m

In [ ]:
fig, m = make_basemap()

kw = {'color': '#FF9900', 'linestyle': '-', 'linewidth': 1.5}
parallels = range(-60, 90, 30)
meridians = range(-360, 360, 60)
m.drawparallels(parallels, labels=[1, 0, 0, 0])
m.drawmeridians(meridians, labels=[0, 0, 1, 0])

df['x'], df['y'] = m(df['lon'].values, df['lat'].values)
m.plot(df['x'].values, df['y'].values, **kw);

In [ ]:
from IPython.display import IFrame

src = 'https://matplotlib.org/basemap/users/intro.html#cartopy-new-management-and-eol-announcement'

IFrame(src, width=700, height=500)

In [ ]:
import cartopy.crs as ccrs


def make_map(projection=ccrs.PlateCarree(), figsize=(5, 5)):
    fig, ax = plt.subplots(
        figsize=figsize,
        subplot_kw={'projection': projection})
    return fig, ax

In [ ]:
import cartopy.feature as cfeature


fig, ax = make_map(projection=ccrs.Robinson(), figsize=(9, 9))

ax.set_global()
ax.coastlines(resolution='110m', color='k')
ax.add_feature(cfeature.LAND, facecolor='0.75')

ax.plot(df['lon'], df['lat'], transform=ccrs.Geodetic(), **kw);

In [ ]:
fig, ax = make_map(projection=ccrs.PlateCarree())
ax.coastlines()

kw = {'linewidth': 4, 'color': 'g', 'transform': ccrs.Geodetic()}
ax.plot([-100, 50], [25, 25], label='Geodetic1', **kw)
ax.plot([-38, 147], [-13, -42], label='Geodetic1', **kw)

kw.update({'color': 'b', 'transform': ccrs.PlateCarree()})
ax.plot([-100, 50], [25, 25], label='PlateCarree1', **kw)
ax.plot([-38, 147], [-13, -42], label='PlateCarree2', **kw)

ax.legend(loc=(1.05, 0.5));

<div class="alert alert-success" style="font-size:80%">
<b>Exercise 00: check what `cartopy` adds to the mpl `ax`.</b><br><br>

tips:

- create a `plt.axes()` object;
- create a new one with a projection;
- compare new methods were added on the second one.

</div>

In [ ]:
%load 00-basemap_cartopy-00-sol.py

<div class="alert alert-info" style="font-size:80%">
<b>REFERENCE</b>: <br><br>

<ul>
  <li>`coastlines`</li>
  <li>`set_global`</li>
  <li>`gridlines`</li>
  <li>`add_feature`</li>
  <li>`set_extent`</li>
  <li>`projection`</li>
  <li>`get_extent`</li>
</ul>

</div>

<div class="alert alert-warning" style="font-size:80%">
<b>REFERENCE</b>: <br><br>

<ul>
  <li>`img_factories`</li>
  <li>`natural_earth_shp`</li>
  <li>`outline_patch`</li>
  <li>`read_user_background_images`</li>
  <li>`set_boundary`</li>
  <li>`tissot`</li>
  <li>`add_geometries`</li>
  <li>`add_raster`</li>
  <li>`add_wms`</li>
  <li>`add_wmts`</li>
  <li>`background_img`</li>
  <li>`background_patch`</li>
  <li>`hold_limits`</li>
</ul>

</div>

In [ ]:
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import (LONGITUDE_FORMATTER,
                                   LATITUDE_FORMATTER)
fig, ax = make_map(projection=ccrs.PlateCarree(), figsize=(3, 3))
ax.stock_img()

coastline = cfeature.GSHHSFeature(scale='coarse')
ax.add_feature(coastline)

gl = ax.gridlines(draw_labels=True)
gl.xlabels_bottom = gl.ylabels_left = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlocator = mticker.FixedLocator([-30, -15, 0, 15, 30, 45, 60])
ax.set_extent([-30, 60, -40, 40])

<div class="alert alert-success" style="font-size:80%">
<b>Exercise 01: repeat the map above and change.</b><br><br>

- the line style: `ax.gridlines(some opts)`
- the y label style: `ylabel_style(dict with text opts)`
</div>

In [ ]:
%load 00-basemap_cartopy-01-sol.py

In [ ]:
from cartopy.io import img_tiles

url = ('https://server.arcgisonline.com/ArcGIS/rest/services/'
       'World_Imagery/MapServer/tile/{z}/{y}/{x}.jpg')

request = img_tiles.GoogleTiles(url)
fig, ax = plt.subplots(figsize=(9, 9),
                       subplot_kw=dict(projection=request.crs))

extent = -48.50, -48.35, -27.35, -27.5
ax.set_extent(extent)

zoom = 13
ax.add_image(request, zoom)

### Natural Earth

In [ ]:
from cartopy.feature import NaturalEarthFeature

fig, ax = make_map(projection=ccrs.LambertConformal(), figsize=(9, 9))

ax.set_extent([-125, -66.5, 20, 50], ccrs.Geodetic())
ax.background_patch.set_visible(False)
ax.outline_patch.set_visible(False)

states = NaturalEarthFeature(category='cultural',
                             scale='110m',
                             facecolor='none',
                             name='admin_1_states_provinces_shp')
feat = ax.add_feature(states, edgecolor='gray')

### Image overlay

In [ ]:
lon = [-40.77, -40.51, -40.30, -40.23, -40.13, -40.06, -39.99,
       -39.87, -39.72, -39.52, -39.32, -39.11, -38.91, -38.71]
lat = [-21.29, -21.39, -21.48, -21.51, -21.56, -21.58, -21.62,
       -21.69, -21.76, -21.86, -21.96, -22.08, -22.15, -22.25]

In [ ]:
fig, ax = make_map(projection=ccrs.PlateCarree(), figsize=(8, 8))

fname = path.joinpath('AVHRR.png')
img = plt.imread(str(fname))
img_extent = (-48, -32, -30, -18)

ax.imshow(img, origin='upper', extent=img_extent,
          transform=ccrs.PlateCarree())

lines = ax.plot(lon, lat, 'blue', marker='*',
                markersize=10, alpha=0.5)

### Shapefiles

In [ ]:
extent = [-39, -38.25, -13.25, -12.5]
coastline = cfeature.GSHHSFeature(scale='full')

fig, ax = make_map(projection=ccrs.PlateCarree())
ax.set_extent(extent)
art = ax.add_feature(coastline)

In [ ]:
import geopandas

fname = path.joinpath('BTS', 'BTS.shp')
gdf = geopandas.read_file(str(fname))

gdf

In [ ]:
fig, ax = make_map(projection=ccrs.PlateCarree())
ax.set_extent(extent)

geometry = gdf['geometry'].iloc[0]
ax.add_geometries([geometry], ccrs.PlateCarree(),
                  facecolor='w', edgecolor='black');

### Re-projecting data

In [ ]:
fname = path.joinpath('rivers_BTS.xlsx')

df = pd.read_excel(fname)

df.head()

In [ ]:
from pyproj import Proj

proj_string = [
    '+proj=utm',
    '+zone=24L',
    '+south',
    '+ellps=WGS84',
    '+datum=WGS84',
    '+units=m',
    '+no_defs',
]
    
utm = Proj(' '.join(proj_string))

df['lon'], df['lat'] = utm(df['x'].values,
                           df['y'].values, inverse=True)
df.head()

In [ ]:
fig, ax = make_map(projection=ccrs.PlateCarree(), figsize=(7, 7))
ax.set_extent(extent)

ax.add_geometries([geometry], ccrs.PlateCarree(),
                  facecolor='w', edgecolor='black')
ax.plot(df['lon'], df['lat'], 'ro', alpha=0.5)

transform = ccrs.UTM(zone=24,
                     southern_hemisphere=True,
                     globe=ccrs.Globe(datum='WGS84', ellipse='WGS84'))

ax.plot(df['x'], df['y'], 'bo', alpha=0.5,
        transform=transform);

# Questions?

![https://xkcd.com/977/](https://imgs.xkcd.com/comics/map_projections.png)